In [1]:
import pandas as pd
import numpy as np

In [2]:
outputs_path="../outputs/"
data_path="../data/"

df_labels_task=pd.read_json(outputs_path+"sg_ie/positives_tasks_methods_clusters_final_f.json")
df_task_test=pd.read_json(outputs_path+"sg_ie/test_scirex_tasks_methods_clusters_final_f.json")


In [3]:
df_labeled=pd.read_csv(outputs_path+"sg_classifier/train_set_labeled_bronze_title_15pct_nk.csv")

In [4]:
total_negatives=pd.read_csv(outputs_path+"sg_classifier/all_negative_examples_final.csv")

In [5]:
total_pos=pd.read_csv(outputs_path+"sg_classifier/all_positive_examples_final.csv")

In [6]:
total_pos.shape

(8703, 7)

In [7]:
df_labeled.loc[df_labeled.ID.isin(df_task_test.ID.unique())]

,ID,title,abstract,title_abstract,label,year,url
385,ostendorff-etal-2020-aspect,Aspect-based Document Similarity for Research ...,Traditional document similarity measures provi...,Aspect-based Document Similarity for Research ...,1,2020,https://aclanthology.org/2020.coling-main.545
386,zhou-etal-2010-exploiting,Exploiting Multi-Features to Detect Hedges and...,"In this paper, we present a machine learning a...",Exploiting Multi-Features to Detect Hedges and...,1,2010,https://aclanthology.org/W10-3015
387,dilsizian-etal-2014-new,A New Framework for Sign Language Recognition ...,Current approaches to sign recognition by comp...,A New Framework for Sign Language Recognition ...,1,2014,http://www.lrec-conf.org/proceedings/lrec2014/...
388,losch-etal-2018-european,{E}uropean Language Resource Coordination: Col...,"In order to help improve the quality, coverage...",European Language Resource Coordination: Colle...,1,2018,https://aclanthology.org/L18-1213
389,koizumi-etal-2002-annotated,An Annotated {J}apanese {S}ign {L}anguage Corpus,Sign language is characterized by its interact...,An Annotated Japanese Sign Language Corpus. Si...,1,2002,http://www.lrec-conf.org/proceedings/lrec2002/...
...,...,...,...,...,...,...,...
7061,owoputi-etal-2013-improved,Improved Part-of-Speech Tagging for Online Con...,We consider the problem of part-of-speech tagg...,Improved Part-of-Speech Tagging for Online Con...,0,2013,https://aclanthology.org/N13-1039.pdf
7062,kang-etal-2020-neural,Neural Mask Generator: Learning to Generate Ad...,We propose a method to automatically generate ...,Neural Mask Generator: Learning to Generate Ad...,0,2020,https://aclanthology.org/2020.emnlp-main.493.pdf
7063,ligozat-2013-question,Question Classification Transfer,Question answering systems have been developed...,Question Classification Transfer. Question ans...,0,2013,https://aclanthology.org/P13-2076.pdf
7064,hozumi-etal-1993-integration,Integration of Morphological and Syntactic Ana...,Morphological analysis of Japanese is very dif...,Integration of Morphological and Syntactic Ana...,0,1993,https://aclanthology.org/1993.iwpt-1.10.pdf


In [8]:
total_negatives.shape

(58941, 7)

In [9]:
58941+5000

63941

In [10]:
df_labels_task.shape

(8133, 3)

In [11]:
test_set=pd.read_csv(outputs_path+"general/test_set_final.csv")
train_set=pd.read_csv(outputs_path+"general/train_set_final.csv")
dev_set=pd.read_csv(outputs_path+"general/dev_set_final.csv")

In [12]:
nlp4sg=pd.concat([train_set,dev_set,test_set])

In [13]:
nlp4sg=nlp4sg.loc[nlp4sg.label==1]

In [14]:
df_labels_task.ID.nunique()

8133

In [15]:
8416-8133

283

In [16]:
nlp4sg.shape

(603, 17)

In [17]:
df_task_test=df_task_test.loc[df_task_test.ID.isin(nlp4sg.ID.unique())]

In [18]:
df_labels_task=pd.concat([df_labels_task,df_task_test]).reset_index(drop=True)

In [19]:
df_labels_task.ID.nunique()

8463

In [20]:
df_labels_task['Method']=[list([]) for _ in range(df_labels_task.shape[0])]
df_labels_task['Task']=[list() for _ in range(df_labels_task.shape[0])]


In [21]:
for i,d in df_labels_task.iterrows():
    tasks=[]
    for e in d['task_scirex']:
        tasks.append(e['top_word'])
    df_labels_task.at[i,'Task']=tasks
    methods=[]
    for e in d['method_scirex']:
        methods.append(e['top_word'])
    df_labels_task.at[i,'Method']=methods

In [22]:
df_labels_task=df_labels_task.explode('Task').rename(columns={'Task':'tasks'})

df_labels_task=df_labels_task.explode('Method').rename(columns={'Method':'methods'})

df_labels_task=df_labels_task.loc[~df_labels_task.tasks.isin(['natural language processing','nlp','nlp applications'])]

df_labels_task=df_labels_task.loc[~df_labels_task.methods.isin(['natural language processing','nlp','nlp applications'])]

df_labels_task.methods=df_labels_task.methods.fillna('no_method')
df_labels_task.tasks=df_labels_task.tasks.fillna('no_task')

df_labels_task['total_methods']=df_labels_task.groupby(['ID']).methods.transform('count')

df_labels_task['weight']=1/df_labels_task.total_methods

In [24]:
###df_mapping=pd.read_csv(outputs+"words_mapping.csv")
df_mapping=pd.read_csv("../sg_match/clusters.psv",sep='|')

df_mapping.center=df_mapping.center.fillna(method='ffill')

df_mapping=df_mapping.loc[df_mapping.word!='Cluster name: ']

df_mapping.word=df_mapping.word.str.rstrip(' ').str.lstrip(' ')
df_mapping.center=df_mapping.center.str.rstrip(' ').str.lstrip(' ')

df_mapping_tasks=df_mapping.rename(columns={'word':'tasks','center':'center_task'})
df_mapping_methods=df_mapping.rename(columns={'word':'methods','center':'center_method'})

df_labels_task=df_labels_task.merge(df_mapping_tasks,on=['tasks'],how='left').merge(df_mapping_methods,on=['methods'],how='left')

df_labels_task=df_labels_task.assign(tasks=np.where((~df_labels_task.center_task.isna()),df_labels_task.center_task,df_labels_task.tasks))

df_labels_task=df_labels_task.assign(methods=np.where((~df_labels_task.center_method.isna()),df_labels_task.center_method,df_labels_task.methods))

In [28]:
df_labels_task.weight.sum()

8446.278571428573

In [25]:
df_labels_task.weight.describe()

count    81286.000000
mean         0.103908
std          0.135970
min          0.005556
25%          0.033333
50%          0.055556
75%          0.111111
max          1.000000
Name: weight, dtype: float64

In [26]:
tasks_count=df_labels_task.loc[:,['ID','tasks']].drop_duplicates()

In [27]:
tasks_count.groupby(['tasks']).count().sort_values('ID',ascending=False).head(10)

,ID
tasks,
machine translation,530
classification,342
no_task,323
toxicity detection,279
annotation,213
named entity recognition,184
sentiment analysis,127
text summarization,125
computational linguistics,117


In [21]:
df_labels_task.groupby(['tasks']).weight.sum().reset_index().sort_values('weight',ascending=False)

,tasks,weight
10590,no_task,323.000000
9074,machine translation,209.191903
6889,hate speech,146.405891
2337,classification,113.442039
561,annotation,67.861086
...,...,...
2757,combining vision,0.041667
13967,spatial language learning and reasoning,0.041667
13968,spatial language understanding,0.041667
13965,spatial information extraction,0.041667


In [30]:
methods_count=df_labels_task.loc[:,['ID','methods']].drop_duplicates()

In [31]:
methods_count.groupby(['methods']).count().sort_values('ID',ascending=False).head(10)

,ID
methods,
no_method,1005
bert,359
machine learning methods,335
classification,251
language models,229
deep neural network,211
machine translation,188
transformers,182
word embeddings,151


In [32]:
methods_count=methods_count.assign(llm=np.where(methods_count.methods.str.lower().str.contains('transformer|language model|bert|gpt|xlnet|electra|t5|bloom|lamda'),'language model',methods_count.methods))

In [33]:
methods_count=methods_count.sort_values('llm').drop_duplicates(subset=['ID','llm'])

In [34]:
methods_count=methods_count.assign(llm=np.where(methods_count.methods=='bert','bert',methods_count.llm))

In [35]:
methods_count=methods_count.drop_duplicates(['ID','llm'])

In [36]:
methods_count.groupby(['llm']).count().sort_values('ID',ascending=False).head(16)

,ID,methods
llm,,
no_method,1005,1005
language model,773,773
machine learning methods,335,335
bert,252,252
classification,251,251
deep neural network,211,211
machine translation,188,188
word embeddings,151,151
support vector machine,136,136


In [ ]:
methods_count=methods_count.assign(llm=np.where(methods_count.methods.str.lower().str.contains('transformer|language model|bert|gpt|xlnet|electra|t5|bloom|lamda'),'language model',methods_count.methods))

In [10]:
df_labels_task.groupby(['methods']).weight.sum().reset_index().sort_values('weight')

,methods,weight
973,aspect based document similarity,0.038462
3547,compilation techniques,0.041667
5011,data structures,0.041667
1151,attribute value logic,0.041667
24356,warrens abstract machine,0.041667
...,...,...
2964,classification,119.533886
1794,bert,122.103022
12870,machine translation,144.750638
12840,machine learning methods,162.749206


In [ ]:
# transfer learning? , attention. mlm

In [34]:
methods_count=methods_count.assign(llm3=np.where(methods_count.methods.str.lower().str.contains('transformer|language model|bert|gpt|xlnet|electra|t5|bloom|lamda'),'language model','other'))

In [28]:
methods_count=methods_count.assign(llm2=np.where(methods_count.llm.str.lower().str.contains('transfer learning|attention|transformer|language model|bert|gpt|xlnet|electra|t5|bloom|lamda'),1,0))

In [29]:
methods_count.loc[methods_count.llm=='language model'].llm2.value_counts()

1    759
Name: llm2, dtype: int64

In [ ]:
methods_count.loc[methods_count.llm=='language model'].llm2.value_counts()

In [35]:
methods_count.drop_duplicates(subset=['ID','llm3']).llm3.value_counts()

other             8324
language model    1025
Name: llm3, dtype: int64

In [37]:
methods_count.drop_duplicates(subset=['ID','llm3']).shape

(9349, 5)

In [30]:
methods_count.drop_duplicates(subset=['ID','llm2']).llm2.value_counts()

0    8303
1    1232
Name: llm2, dtype: int64

In [38]:
df_labels_task.ID.nunique()

8416

In [39]:
1025/8416

0.12179182509505704

In [40]:
1-(1025/8416)

0.878208174904943

In [50]:
df_labels_task=df_labels_task.assign(llm=np.where(df_labels_task.methods.str.lower().str.contains('transfer learning|attention|transformer|language model|bert|gpt|xlnet|electra|t5|bloom|lamda'),1,0))

In [51]:
df_labels_task.drop_duplicates(subset=['ID']).llm.value_counts()

0    12107
1      763
Name: llm, dtype: int64

In [52]:
763/(12107+763)

0.05928515928515928

In [26]:
612/(12258+612)

0.04755244755244755

In [13]:
df_labels_task.loc[df_labels_task.methods.str.lower().str.contains('bert|gpt|xlnet|electra|t5')].methods.unique()

array(['roberta', 'bert', 'bert baseline classifier',
       'mbert transformer model', 'multi depth distilbert',
       'domain specific language model scibert', 'graphbert model',
       'bert base model', 'bert representations',
       'gpt 2 fine tuned models',
       'bert large cased question answering model',
       'xlnet natural language inference model', 'xlnet', 'gpt',
       'bert based adverse drug effect tweet classification',
       'bert based hierarchical aggregation model', 'bert classifiers',
       'bert based architectures', 'bert based multilingual model',
       'vl bert', 'bert multilingual model', 'bert language model',
       'covid twitter bert', 'bert - based classifier',
       'fine tuned bert bidirectional encoder representations',
       'bert based dual encoder model', 'roberta base model',
       'roberta pre trained model', 'biobert baseline',
       'bert based classification model', 'romanian bert model',
       'jurbert', 'bert and derivative model

In [49]:
others=df_labels_task.loc[~df_labels_task.methods.str.lower().str.contains('transformer|language model|bert|gpt|xlnet|electra|t5')]

In [44]:
others=others.groupby(['methods']).weight.sum().reset_index().sort_values('weight',ascending=False)

In [48]:
list(others.head(500).methods.unique())

['no_method',
 'machine learning methods',
 'machine translation',
 'classification',
 'deep neural network',
 'word embeddings',
 'annotation',
 'nlp applications',
 'support vector machine',
 'neural network',
 'conditional random field',
 'lstm',
 'neural models',
 'language technology',
 'unsupervised algorithm',
 'convolutional neural network',
 'supervised learning',
 'computational models',
 'recurrent neural networks',
 'multi task learning',
 'transfer learning',
 'named entity recognition',
 'ensemble methods',
 'topic models',
 'part of speech',
 'parser',
 'attention',
 'domain adaption',
 'wordnet',
 'feature engineering',
 'logistic regression',
 'automatic speech recognition',
 'umls',
 'search engine',
 'dialogue system',
 'rule based system',
 'evaluation methods',
 'error analysis',
 'pretrained models',
 'fact checking',
 'grammar',
 'hybrid approach',
 'computational model',
 'sentiment analysis',
 'fine tuning',
 'active learning',
 'statistical analysis',
 'probab

In [30]:
others.index('drugbank')

999

In [37]:
others[2999:]

['auto encoder based differentially private text transformation',
 'mia attacks',
 'membership inference attacks',
 'transformation process',
 'supervised and semi supervised approaches',
 'nearly unsupervised hashcode representations',
 'kernelized locality sensitive hashcodes',
 'sign language translation system',
 'bigram models of non manual features',
 'sequence classification',
 'unigram',
 'bilingual sign language dictionary',
 'www',
 'class classification',
 'supervised and unsupervised methods',
 'stock investment taxonomy',
 'companion teaching',
 'dialogue policy',
 'initial policy',
 'bigram',
 'k nn',
 'nave bayes',
 'emotion words',
 'classification systems',
 'graph memory networks',
 'training pipeline',
 'software systems',
 'pyreval',
 'content models',
 'natural language processing software',
 'machine aids',
 'grader ensemble',
 'localization processes',
 'runet',
 'open source framework',
 'mudes',
 'automatic detection methods',
 'automated annotation scheme lear